In [1]:
#프로그램 6-1 pyqt5로 간단한 gui 만들기(버튼 클릭하면 삐 소리 들려주기)

from PyQt5.QtWidgets import *
import sys
import winsound

class BeepSound(QMainWindow):  #BeepSound 클래스 선언
    def __init__(self):                #생성자 함수
        super().__init__()
        self.setWindowTitle("삑 소리 내기")            #윈도우 이름과 위치 지정
        self.setGeometry(200, 200, 500, 100)

        shortBeepButton = QPushButton("짧게", self)       #버튼 생성
        longBeepButton = QPushButton("길게", self)
        quitButton = QPushButton("종료", self)
        self.label = QLabel("환영합니다!", self)

        shortBeepButton.setGeometry(10, 10, 100, 30)     #버튼 위치와 크기 조정
        longBeepButton.setGeometry(110, 10, 100, 30)
        quitButton.setGeometry(210, 10, 100, 30)
        self.label.setGeometry(10, 50, 400, 30)

        shortBeepButton.clicked.connect(self.shortBeepFunction)        #콜백함수 지정, Short Beep Button이 눌리면 function 실행
        longBeepButton.clicked.connect(self.longBeepFunction)
        quitButton.clicked.connect(self.quitFunction)

    def shortBeepFunction(self):
        self.label.setText("주파수 1000으로 0.5초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 500)

    def longBeepFunction(self):
        self.label.setText("주파수 1000으로 3초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 3000)

    def quitFunction(self):
        self.close()

app = QApplication(sys.argv)
win = BeepSound()                  #beeo sound 클래스의 객체 생상
win.show()
app.exec_()

0

In [ ]:
#프로글매 6-2 
from PyQt5.QtWidgets import *   
import sys                     
import cv2 as cv               

# Video 클래스 선언 (QMainWindow 상속)
class Video(QMainWindow):
    def __init__(self):       # 생성자 함수
        super().__init__()    # 부모 클래스 초기화

        self.setWindowTitle('비디오에서 프레임 추출')     # 윈도우 이름과 위치 지정
        self.setGeometry(200, 200, 500, 100)        
        
        # 버튼 생성
        videoButton = QPushButton('비디오 보기', self)
        captureButton = QPushButton('프레임 잡기', self)
        saveButton = QPushButton('프레임 저장', self)
        quitButton = QPushButton('나가기', self)

        # 버튼 위치와 크기 설정 
        videoButton.setGeometry(10, 10, 100, 30)
        captureButton.setGeometry(110, 10, 100, 30)
        saveButton.setGeometry(210, 10, 100, 30)
        quitButton.setGeometry(310, 10, 100, 30)

        # 콜백함수 지정
        videoButton.clicked.connect(self.videoFunction)        # 비디오 보기 버튼
        captureButton.clicked.connect(self.captureFunction)    # 프레임 잡기 버튼
        saveButton.clicked.connect(self.saveFunction)          # 프레임 저장 버튼
        quitButton.clicked.connect(self.quitFunction)          # 나가기 버튼
        
    def videoFunction(self):
        self.cap=cv.VideoCapture(0,cv.CAP_DSHOW)   #카메라와 연결 시도
        if not self.cap.isOpened():self.close()
            
        while True:
            ret,self.frame=self.cap.read()
            if not ret:break
            cv.imshow('video display', self.frame)
            cv.waitKey(1)
                
    def captureFunction(self):
        self.capturedFrame=self.frame
        cv.imshow('Captured Frame', self.capturedFrame)
        
    def saveFunction(self):       #파일저장
        fname=QFileDialog.getSaveFileName(self, '파일저장', './')
        cv.imwrite(fname[0], self.capturedFrame)
        
    def quitFunction(self):                       #카메라와 연결 끊음 
        self.cap.release()
        cv.destroyAllWindows()
        self.close()
        
app = QApplication(sys.argv)
win = Video()
win.show()
app.exec_()

In [1]:
#프로그램6-3 Grabcut을 이용해 관심 물체 오리기

from PyQt5.QtWidgets import *   
import sys                     
import cv2 as cv      
import numpy as np

class Orim(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('오림')
        self.setGeometry(200, 200, 700, 200)
        
        fileButton=QPushButton('파일', self)
        paintButton=QPushButton('페인팅', self)
        cutButton=QPushButton('오림', self)
        incButton=QPushButton('+', self)
        decButton=QPushButton('-', self)
        saveButton=QPushButton('저장', self)
        quitButton=QPushButton('나가기', self)
        
        fileButton.setGeometry(10,10,100,30)
        paintButton.setGeometry(110,10,100,30)
        cutButton.setGeometry(210,10,100,30)
        incButton.setGeometry(310,10,50,30)
        decButton.setGeometry(360,10,50,30)
        saveButton.setGeometry(410,10,100,30)
        quitButton.setGeometry(510,10,100,30)
        
        fileButton.clicked.connect(self.fileOpenFunction)
        paintButton.clicked.connect(self.paintFunction)
        cutButton.clicked.connect(self.cutFunction)
        incButton.clicked.connect(self.incFunction)
        decButton.clicked.connect(self.decFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)
        
        self.BrushSize=5
        self.LColor, self.Rcolor=(255, 0,0), (0,0,255)
        
    def fileOpenFunction(self):
        fname=QFileDialog.getOpenFileName(self, 'Open file>,', './')
        self.img=cv.imread(fname[0])
        if self.img is None: sys.exit('파일을 찾을 수 없습니다')
            
        self.img_show=np.copy(self.img)            #표시용영상
        cv.imshow('painting', self.img_show)
        
        self.mask=np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:,:]=cv.GC_PR_BGD         #모든 화소를 배경일 것 같음으로 초기화
        
    def paintFunction(self):
        cv.setMouseCallback('painting', self.painting)
        
    def painting(self,event,x,y,flags,param):
        if event==cv.EVENT_LBUTTONDOWN:
            cv.circle(self.img_show, (x,y), self.BrushSize,self.LColor,-1)       #왼쪽버튼을 클릭하면 파란색
            cv.circle(self.mask, (x,y), self.BrushSize, cv.GC_FGD,-1)
        elif event==cv.EVENT_RBUTTONDOWN:
            cv.circle(self.img_show, (x,y), self.BrushSize, self.RColor,-1)        #오른쪽버튼을 클릭하면  빩간색
            cv.circle(self.mask, (x,y), self.BrushSize, cv.GC_BGD,-1)
                      
        elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_LBUTTON:
            cv.circle(self.img_show, (x,y), self.BrushSize, self.LColor,-1)        #왼쪽버튼을 클릭하고 이동하면 v파란색
            cv.circle(self.mask, (x, y), self.BrushSize, cv.GC_FGD,-1)
                      
        elif event==cv.EVENT_MOUSEMOVE and flags==cv.EVENT_FLAG_RBUTTON:
            cv.circle(self.img_show, (x,y), self.BrushSize, self.RColor,-1)        #오른쪽버튼을 클릭하고 이동하면 빨간색
            cv.circle(self.mask, (x,y), self.BrushSize, cv.GC_BGD,-1)
                      
        cv.imshow('painting', self.img_show)
                      
    def cutFunction(self):
        background = np.zeros((1, 65), np.float64)
        foreground = np.zeros((1, 65), np.float64)
        cv.grabCut(self.img, self.mask, None, background, foreground, 5, cv.GC_INIT_WITH_MASK)
        mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype('uint8')
        self.grabImg = self.img * mask2[:, :, np.newaxis]
        cv.imshow('Scissoring', self.grabImg)

    def incFunction(self):
        self.BrushSize = min(20, self.BrushSize + 1)

    def decFunction(self):
        self.BrushSize = max(1, self.BrushSize - 1)

    def saveFunction(self):
        fname = QFileDialog.getSaveFileName(self, '파일 저장', './')
        cv.imwrite(fname[0], self.grabImg)

    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()
                      
app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()

0

In [1]:
#6-4 교통약자 보호구역 알림 구현하기

import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys
import winsound

class TrafficWeak(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('교통약자보호')
        self.setGeometry(200,200,700,200)
        
        signButton=QPushButton('표지판 등록', self)
        roadButton=QPushButton('도로 영상 불러옴', self)
        recognitionButton=QPushButton('인식', self)
        quitButton=QPushButton('나가기', self)
        self.label=QLabel('환영합니다', self)
        
        signButton.setGeometry(10,10,100,30)
        roadButton.setGeometry(100,10,100,30)
        recognitionButton.setGeometry(210,10,100,30)
        quitButton.setGeometry(510,10,100,30)
        self.label.setGeometry(10,40,600,170)
        
        signButton.clicked.connect(self.signFunction)
        roadButton.clicked.connect(self.roadFunction)
        recognitionButton.clicked.connect(self.recognitionFunction)
        quitButton.clicked.connect(self.quitFunction)
        
        self.signFiles=[['child.png', '어린이'], ['elder.png', '노인'], ['disabled.png', '장애인']]   #표지판 모델 영상
        self.signImgs=[]
        
    def signFunction(self):
        self.label.clear()
        self.label.setText('교통약자 표지판을 등록합니다')
        
        for fname,_ in self.signFiles:
            self.signImgs.append(cv.imread(fname))
            cv.imshow(fname, self.signImgs[-1])
            
    def roadFunction(self):
        if self.signImgs==[]:
            self.label.setText('먼저 표지판을 등록하세요.')
        else:
            fname=QFileDialog.getOpenFileName(self,'파일읽기', './')
            self.roadImg=cv.imread(fname[0])
            if self.roadImg is None: sys.exit('파일을 찾을 수 없습니다.')
                
            cv.imshow('Road scene', self.roadImg)
            
    def recognitionFunction(self):
        if self.roadImg is None:
            self.label.setText('먼저 도로 영상을 입력하세요.')
        else:
            sift=cv.SIFT_create()
            
            KD=[]
            for img in self.signImgs:
                gray=cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                KD.append(sift.detectAndCompute(gray, None))
                
            grayRoad=cv.cvtColor(self.roadImg, cv.COLOR_BGR2GRAY) #명암으로 변환
            road_kp, road_des=sift.detectAndCompute(grayRoad,None)  #키포인트와 기술자 추출
            
            matcher=cv.DescriptorMatcher_create(cv.DescriptorMatcher_FLANNBASED)
            GM=[]              #여러표지판 영상의 good match를 저장
            for sign_kp,sign_des in KD:
                knn_match=matcher.knnMatch(sign_des, road_des, 2)
                T=0.7
                good_match=[]
                for nearest1, nearest2 in knn_match:
                    if (nearest1.distance/nearest2.distance)<T:
                        good_match.append(nearest1)
                GM.append(good_match)
                
            best=GM.index(max(GM,key=len))   #매칭 쌍 개수가 최대인 표지판 찾기
            
            img_match = cv.drawMatches(self.signImgs[best], KD[best][0], self.roadImg, road_kp, GM[best], None)
            cv.imshow('Matches and Homography', img_match)
            
            self.label.setText(self.signFiles[best][1]+'보호구역입니다. 30km로 서행하세요')
            winsound.Beep(3000,500)
        
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()
        
app=QApplication(sys.argv)
win=TrafficWeak()
win.show()
app.exec_()

0

In [1]:
#프로그램 6-5 비디오에서 수집한 영상을 봉합하여 파노라마 영상 제작하기

from PyQt5.QtWidgets import *
import cv2 as cv
import numpy as np
import winsound
import sys

class Panorama(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('파노라마 영상')
        self.setGeometry(200, 200, 700, 200)
        
        collectButton=QPushButton('영상수집', self)
        self.showButton=QPushButton('영상보기', self)
        self.stitchButton=QPushButton('봉합', self)
        self.saveButton=QPushButton('저장', self)
        quitButton=QPushButton('나가기', self)
        self.label=QLabel('환영합니다!', self)
        
        collectButton.setGeometry(10,25,100,30)
        self.showButton.setGeometry(110, 25, 100, 30)
        self.stitchButton.setGeometry(210, 25, 100, 30)
        self.saveButton.setGeometry(310, 25, 100, 30)
        quitButton.setGeometry(450, 25, 100, 30)
        self.label.setGeometry(10, 70, 600, 170)
        
        self.showButton.setEnabled(False)
        self.stitchButton.setEnabled(False)
        self.saveButton.setEnabled(False)
        
        collectButton.clicked.connect(self.collectFunction)
        self.showButton.clicked.connect(self.showFunction)
        self.stitchButton.clicked.connect(self.stitchFunction)
        self.saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)
        
    def collectFunction(self):
        self.showButton.setEnabled(False)
        self.stitchButton.setEnabled(False)
        self.saveButton.setEnabled(False)
        self.label.setText('c를 여러번 눌러 수집하고 끝나면 q를 눌러 비디오를 끕니다. ')
        
        self.cap=cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():sys.exit('카메라 연결 실패')
            
        self.imgs=[]
        while True:
            ret, frame=self.cap.read()
            if not ret: break
            
            cv.imshow('video display', frame)
            
            key=cv.waitKey(1)
            if key==ord('c'):
                self.imgs.append(frame) #영상저장
            elif key==ord('q'):
                self.cap.release()
                cv.destroyWindow('video display')
                break
                
        if len(self.imgs)>=2:  #수집한 영상이 두장 이상이면
            self.showButton.setEnabled(True)
            self.stitchButton.setEnabled(True)
            self.saveButton.setEnabled(True)
    
    def showFunction(self):
        self.label.setText('수집된 영상은'+str(len(self.imgs))+'장입니다.')
        stack = cv.resize(self.imgs[0], dsize=(0,0), fx=0.25, fy=0.25)
        for i in range(1, len(self.imgs)):
            stack=np.hstack((stack, cv.resize(self.imgs[i], dsize=(0,0), fx=0.25, fy=0.25)))
        cv.imshow('Image collection', stack)
        
    def stitchFunction(self):
        stitcher=cv.Stitcher_create()
        status, self.img_stitched=stitcher.stitch(self.imgs)
        if status == cv.STITCHER_OK:
            cv.imshow('Image stitched panorama', self.img_stitched)
        else:
            winsound.Beep(3000, 500)
            self.label.setText('파노라마 제작에 실패했습니다. 다시 시도하세요')
            
    def saveFunction(self):
        fname=QFileDialog.getSaveFileName(self, '파일저장', './')
        cv.imwrite(fname[0], self.img_stitched)
        
    def quitFunction(self):
        self.cap.release()
        cv.destroyAllWindows()
        self.close()
        
app=QApplication(sys.argv)
win=Panorama()
win.show()
app.exec_()

0

In [1]:
#6-6  사진영상에 특수효과 처리하기
import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys

class SpecialEffect (QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('사진 특수 효과')
        self.setGeometry(200,200,800,200)
        
        pictureButton=QPushButton('사진 읽기', self)
        embossButton=QPushButton('엠보싱', self)
        cartoonButton=QPushButton('카툰', self)
        sketchButton=QPushButton('연필스케치', self)
        oilButton=QPushButton('유화', self)
        saveButton=QPushButton('저장하기', self)
        self.pickCombo=QComboBox(self)
        self.pickCombo.addItems(['엠보싱', '카툰', '연필 스케치(명암)', '연필 스케치(컬러)',
'유화'])
        quitButton=QPushButton('나가기', self)
        self.label=QLabel('환영합니다', self)
    

        pictureButton.setGeometry (10,10,100,30)
        embossButton.setGeometry(110,10,100,30)
        cartoonButton.setGeometry (210,10,100,30)
        sketchButton.setGeometry(310,10,100,30)
        oilButton.setGeometry (410,10,100,30)
        saveButton.setGeometry (510,10,100,30)
        self.pickCombo.setGeometry (510,40,110,30)
        quitButton.setGeometry (620,10,100,30)
        self.label.setGeometry (10,40,500,170)
        
        pictureButton.clicked.connect(self.pictureOpenFunction)
        embossButton.clicked.connect(self.embossFunction)
        cartoonButton.clicked.connect(self.cartoonFunction)
        sketchButton.clicked.connect(self.sketchFunction) 
        oilButton.clicked.connect(self.oilFunction)
        saveButton.clicked.connect(self.saveFunction)
        quitButton.clicked.connect(self.quitFunction)


    def pictureOpenFunction(self):
        fname=QFileDialog.getOpenFileName(self,'사진읽기','./')
        self.img=cv.imread(fname[0])
        if self.img is None: sys.exit('파일을 찾을 수 없습니다.')
            
        cv.imshow('Painting',self.img)
        
    def embossFunction(self):
        femboss=np.array([[-1.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
        
        gray=cv.cvtColor(self.img, cv.COLOR_BGR2GRAY)
        gray16=np.int16(gray)
        self.emboss=np.uint8 (np.clip (cv.filter2D(gray16,-1, femboss)+128,0,255))

        cv.imshow('Emboss', self.emboss)

    def cartoonFunction(self):
        self.cartoon=cv.stylization (self.img,sigma_s=60, sigma_r=0.45)
        cv.imshow('Cartoon', self.cartoon)

    def sketchFunction(self):
        self.sketch_gray,self.sketch_color=cv.pencilSketch(self.img,sigma_s=60,sigma_r=0.07, shade_factor=0.02)
        cv.imshow('Pencil sketch(gray)', self.sketch_gray)
        cv.imshow('Pencil sketch(color)', self.sketch_color)
                  
    def oilFunction(self):
        self.oil=cv.xphoto.oilPainting(self.img, 10, 1, cv.COLOR_BGR2Lab)
        cv.imshow('Oil painting', self.oil)
                  
    def saveFunction(self):
        fname=QFileDialog.getSaveFileName(self, '파일저장', './')
        
        i=self.pickCombo.currentIndex()
        if i == 0: cv.imwrite(fname[0], self.emboss)
        elif i==1: cv.imwrite(fname[0], self.cartoon)
        elif i==2: cv.imwrite(fname[0], self.sketch_gray)
        elif i==3: cv.imwrite(fname[0], self.sketch_color)
        elif i==4: cv.imwrite(fname[0], self.oil)
    
    def quitFunction(self):
        cv.destroyAllWindows()
        self.close()

app=QApplication(sys.argv)
win=SpecialEffect()
win.show()
app.exec_()

AttributeError: module 'cv2' has no attribute 'xphoto'

0

In [3]:
pip install opencv-contrib-python

   ---------------------------------------- 0.0/46.2 MB ? eta -:--:--
   ---------------------------------------- 0.4/46.2 MB 7.6 MB/s eta 0:00:07
    --------------------------------------- 0.9/46.2 MB 9.0 MB/s eta 0:00:06
   - -------------------------------------- 1.4/46.2 MB 10.7 MB/s eta 0:00:05
   - -------------------------------------- 2.0/46.2 MB 10.6 MB/s eta 0:00:05
   -- ------------------------------------- 2.5/46.2 MB 10.5 MB/s eta 0:00:05
   -- ------------------------------------- 3.0/46.2 MB 10.5 MB/s eta 0:00:05
   -- ------------------------------------- 3.5/46.2 MB 10.5 MB/s eta 0:00:05
   --- ------------------------------------ 3.9/46.2 MB 10.5 MB/s eta 0:00:05
   --- ------------------------------------ 4.4/46.2 MB 10.5 MB/s eta 0:00:04
   ---- ----------------------------------- 4.9/46.2 MB 10.5 MB/s eta 0:00:04
   ---- ----------------------------------- 5.4/46.2 MB 10.4 MB/s eta 0:00:04
   ----- ---------------------------------- 5.9/46.2 MB 10.4 MB/s eta 0:00

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\ProgramData\\Anaconda3\\Lib\\site-packages\\cv2\\cv2.pyd'
Consider using the `--user` option or check the permissions.


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
#프로그램 6-7 비디오 영상에 특수 효과 처리하기

import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys

class VideoSpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('비디오 특수 효과')
        self.setGeometry(200, 200, 400, 100)
        
        videoButton=QPushButton('비디오 시작', self)
        self.pickCombo=QComboBox(self)
        self.pickCombo.addItems(['엠보싱', '카툰', '연필스케치(명암)', '연필스케치(컬러)', '유화'])
        quitButton=QPushButton('나가기', self)
        
        videoButton.setGeometry(10,10,140,30)
        self.pickCombo.setGeometry(150, 10, 110, 30)
        quitButton.setGeometry(280, 10, 100, 30)
        
        videoButton.clicked.connect(self.videoSpecialEffectFunction)
        quitButton.clicked.connect(self.quitFunction)
        
    def videoSpecialEffectFunction(self):
        self.cap=cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():sys.exit('카메라연결실패')
        
        while True:
            ret,frame=self.cap.read()
            if not ret: break
                
            pick_effect=self.pickCombo.currentIndex()
            if pick_effect==0:
                femboss=np.array([[-1.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
        
                gray=cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                gray16=np.int16(gray)
                self.emboss=np.uint8 (np.clip (cv.filter2D(gray16,-1, femboss)+128,0,255))
            
            elif pick_effect==1:
                special_img = cv.stylization(frame, sigma_s=60, sigma_r=0.45)
            elif pick_effect==2:
                special_img, _=cv.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.02)
            elif pick_effect==3:
                _,special_img=cv.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.02)  
            elif pick_effect==4:
                special_img=cv.xphoto.oilPainting(frame, 10, 1, cv.COLOR_BGR2_Lab)
                
            cv.imshow('Special effect', special_img)
            cv.waitKey(1)
            
    def quitFunction(self):
        self.cap.release()
        cv.destroyAllWindows()
        self.close()
        
app=QApplication(sys.argv)
win=VideoSpecialEffect()
win.show()
app.exec_()        
        

UnboundLocalError: local variable 'special_img' referenced before assignment

KeyboardInterrupt: 

0

In [1]:
import cv2 as cv
import numpy as np
from PyQt5.QtWidgets import *
import sys

class VideoSpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle('비디오 특수 효과')
        self.setGeometry(200, 200, 400, 100)
        
        videoButton = QPushButton('비디오 시작', self)
        self.pickCombo = QComboBox(self)
        self.pickCombo.addItems(['엠보싱', '카툰', '연필스케치(명암)', '연필스케치(컬러)', '유화'])
        quitButton = QPushButton('나가기', self)
        
        videoButton.setGeometry(10, 10, 140, 30)
        self.pickCombo.setGeometry(150, 10, 110, 30)
        quitButton.setGeometry(280, 10, 100, 30)
        
        videoButton.clicked.connect(self.videoSpecialEffectFunction)
        quitButton.clicked.connect(self.quitFunction)

    def videoSpecialEffectFunction(self):
        self.cap = cv.VideoCapture(0, cv.CAP_DSHOW)
        if not self.cap.isOpened():
            sys.exit('카메라 연결 실패')

        while True:
            ret, frame = self.cap.read()
            if not ret:
                break

            pick_effect = self.pickCombo.currentIndex()

            if pick_effect == 0:  # 엠보싱
                femboss = np.array([[-1.0, 0.0, 0.0],
                                    [0.0,  0.0, 0.0],
                                    [0.0,  0.0, 1.0]])
                gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                gray16 = np.int16(gray)
                emboss = np.uint8(np.clip(cv.filter2D(gray16, -1, femboss) + 128, 0, 255))
                special_img = cv.cvtColor(emboss, cv.COLOR_GRAY2BGR)

            elif pick_effect == 1:  # 카툰
                special_img = cv.stylization(frame, sigma_s=60, sigma_r=0.45)

            elif pick_effect == 2:  # 연필 스케치 (명암)
                sketch_gray, _ = cv.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.02)
                special_img = cv.cvtColor(sketch_gray, cv.COLOR_GRAY2BGR)

            elif pick_effect == 3:  # 연필 스케치 (컬러)
                _, sketch_color = cv.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.02)
                special_img = sketch_color

            elif pick_effect == 4:  # 유화
                if hasattr(cv, 'xphoto') and hasattr(cv.xphoto, 'oilPainting'):
                    special_img = cv.xphoto.oilPainting(frame, 10, 1, cv.COLOR_BGR2Lab)
                else:
                    special_img = frame
                    print("⚠ 유화 필터를 사용하려면 opencv-contrib-python이 설치되어 있어야 합니다.")

            else:
                special_img = frame  # 예외 처리용 기본 이미지

            cv.imshow('Special Effect', special_img)
            if cv.waitKey(1) & 0xFF == 27:  # ESC 키 종료
                break

        self.cap.release()
        cv.destroyAllWindows()

    def quitFunction(self):
        if hasattr(self, 'cap'):
            self.cap.release()
        cv.destroyAllWindows()
        self.close()


app = QApplication(sys.argv)
win = VideoSpecialEffect()
win.show()
app.exec_()


0

In [1]:
pip install opencv-contrib-python


  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl (46.2 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
